![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FApplied+ML%2FForecasting&file=BigQuery+ML+For+Hierarchical+Forecasting.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20ML/Forecasting/BigQuery%20ML%20For%20Hierarchical%20Forecasting.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FApplied%2520ML%2FForecasting%2FBigQuery%2520ML%2520For%2520Hierarchical%2520Forecasting.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20ML/Forecasting/BigQuery%20ML%20For%20Hierarchical%20Forecasting.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Applied%20ML/Forecasting/BigQuery%20ML%20For%20Hierarchical%20Forecasting.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# BigQuery ML For Hierarchical Forecasting


- Introduce hierarchical forecasting topic
- Data description and prep
- Forecasting the lowest level series
- Bottom-Up Hierarchical Forecasting with built-in functionality
    - Detailed review of the bottom-up approach
- Top-Down Hierarchical Forecasting: Step-by-step instructions
- Top-Down Hierarchical Forecasting: Workflow With BigQuery Procedural Language


Key Links To Document:
- https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-time-series
- https://cloud.google.com/bigquery/docs/arima-time-series-forecasting-with-hierarchical-time-series
- https://cloud.google.com/bigquery/docs/e2e-journey
- https://otexts.com/fpp3/single-level.html


**Data Source:**

This workflow uses data collected from products being sold in stores.  It could be used to forecast product demand at the store level as well as considering the hierarcy of `State | County | City | Store`.  The data are found in the BigQuery Public dataset at:

- `bigquery-public-data.iowa_liquor_sales.sales`

---
## Colab Setup

To run this notebook in Colab run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
    print('Colab authorized to GCP')
except Exception:
    print('Not a Colab Environment')
    pass

Not a Colab Environment


---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-bigquery'),
    ('plotly', 'plotly'),
    ('kaleido', 'kaleido')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [4]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

Inputs

In [5]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [6]:
REGION = 'us-central1'
SERIES = 'applied-ml-forecasting'
EXPERIMENT = 'bqml-hierarchical'

Packages

In [7]:
from google.cloud import bigquery

Clients

In [8]:
# bigquery client
bq = bigquery.Client(project = PROJECT_ID)

# bigquery cell magics
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


Prepare the code below for your environment.

This notebook takes advantage of the [BigQuery IPython magic](https://cloud.google.com/python/docs/reference/bigquery/latest/magics) for legibility and ease of copy/pasting to BigQuery SQL editor. If this notebook is being used from an environment that can run notebooks it needs further preparation: Colab, Colab Enterprise, Vertex AI Workbench Instances, or BigQuery Studio with a Python Notebook. The SQL code in these cells uses the fully qualified [BigQuery table](https://cloud.google.com/bigquery/docs/tables-intro) names in the form `projectname.datasetname.tablename`. Prepare for your environment by:

- Edit > Find
    - Find: `statmike-mlops-349915`
    - Replace: `<your project id>`
    - Replace All

---
## BigQuery Source Data

This workflow uses data collected from products being sold in stores.  It could be used to forecast product demand at the store level as well as considering the hierarcy of `State | County | City | Store`.  The data are found in the BigQuery Public dataset at:

- `bigquery-public-data.iowa_liquor_sales.sales`

In [9]:
%%bigquery
SELECT *
FROM `bigquery-public-data.iowa_liquor_sales.sales`
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,...,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,RINV-05110400027,2024-01-30,2647,HY-VEE #7 / CEDAR RAPIDS,5050 EDGEWOOD RD,CEDAR RAPIDS,52411.0,POINT(-91.698522983 42.029484381),None,LINN,...,43120,BACARDI SUPERIOR PET,6,1750,15.50,23.25,-48,-1116.00,-84.0,-22.19
1,RINV-05297200096,2024-06-11,2621,HY-VEE FOOD STORE #3 / SIOUX CITY,3301 GORDON DR,SIOUX CITY,51105.0,POINT(-96.362866022 42.488984001),None,WOODBURY,...,36908,MCCORMICK 80PRF VODKA PET,6,1750,8.24,12.36,-12,-148.32,-21.0,-5.54
2,RINV-05455500010,2024-10-03,2643,HY-VEE WINE AND SPIRITS / WATERLOO,2126 KIMBALL AVE,WATERLOO,50701.0,POINT(-92.35698 42.47029),None,BLACK HAWK,...,64865,FIREBALL CINNAMON WHISKEY PET,12,750,9.00,13.50,-24,-324.00,-18.0,-4.75
3,RINV-04806800054,2023-08-16,5145,SOUTH SIDE FOOD MART,1101 ARMY POST RD. SUITE A & B,DES MOINES,50315.0,POINT(-93.628625001 41.526920009),None,POLK,...,31470,NEW AMSTERDAM GIN,12,1000,7.83,11.75,-12,-141.00,-12.0,-3.17
4,RINV-05394100050,2024-08-20,2606,HY-VEE WINE AND SPIRITS / HUMBOLDT,1011 13TH ST NORTH,HUMBOLDT,50548.0,POINT(-94.226730035 42.733207011),None,HUMBOLDT,...,35917,FIVE O'CLOCK VODKA,12,1000,4.66,6.99,-12,-83.88,-12.0,-3.17


### Describe Data with `ML.DESCRIBE_DATA`

Reviewing a few records, like above, gives a good sense of how the data is arranged. Before proceeding with machine learning techniques it is important to understand more about these raw columns.  Are they ready to use a features in a model or is some form of feature engineering needed first?  For this, the distribution of values is an important starting point.  

While SQL could be used to look at the distribution, it would be a time consuming process and requires different techniques for different data types like numerical, string, boolean, dates, times, array and struct version of these, and arrays of structs.

To make this process fast and simple, the new [`ML.DESCRIBE_DATA`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-describe-data) function is used to get a single row for each column the describes the data distribution:
- `top_k`: get the top 3 most frequent categories for string columns (default = 1)
- `num_quantiles`: get 4 quantiles for numerical columns (default = 2)

In [10]:
%%bigquery
SELECT *
FROM ML.DESCRIBE_DATA(
    (SELECT * EXCEPT(store_location) FROM `bigquery-public-data.iowa_liquor_sales.sales`),
    STRUCT(3 AS top_k, 4 AS num_quantiles)
)

Query is running:   0%|          |

Downloading:   0%|          |

,name,num_rows,num_nulls,num_zeros,min,max,mean,stddev,median,quantiles,unique,avg_string_length,num_values,top_values,min_array_length,max_array_length,avg_array_length,total_array_length,array_length_quantiles,dimension
0,address,31339341,83720,<NA>,1 E MAIN ST,PO BOX 261 310 W DILLON,NaN,NaN,NaN,[],3199,16.002626,31255621,"[{'value': '3221 SE 14TH ST', 'count': 345286}...",<NA>,<NA>,NaN,<NA>,[],<NA>
1,bottle_volume_ml,31339341,0,10,0,378000,870.840142,619.444568,750.00,"[0.0, 750.0, 750.0, 1000.0, 378000.0]",<NA>,NaN,31339341,[],<NA>,<NA>,NaN,<NA>,[],<NA>
2,bottles_sold,31339341,0,9,-768,15000,10.915819,30.769464,6.00,"[-768.0, 3.0, 6.0, 12.0, 15000.0]",<NA>,NaN,31339341,[],<NA>,<NA>,NaN,<NA>,[],<NA>
3,category,31339341,16974,<NA>,1011000.0,1901200.0,NaN,NaN,NaN,[],114,9.000000,31322367,"[{'value': '1031100.0', 'count': 3344482}, {'v...",<NA>,<NA>,NaN,<NA>,[],<NA>
4,category_name,31339341,25040,<NA>,100 PROOF VODKA,WHITE RUM,NaN,NaN,NaN,[],103,17.457523,31314301,"[{'value': 'AMERICAN VODKAS', 'count': 3250690...",<NA>,<NA>,NaN,<NA>,[],<NA>
5,city,31339341,83719,<NA>,ACKLEY,ZWINGLE,NaN,NaN,NaN,[],501,9.193062,31255622,"[{'value': 'DES MOINES', 'count': 2664255}, {'...",<NA>,<NA>,NaN,<NA>,[],<NA>
6,county,31339341,160522,<NA>,ADAIR,WRIGHT,NaN,NaN,NaN,[],100,6.428723,31178819,"[{'value': 'POLK', 'count': 5831419}, {'value'...",<NA>,<NA>,NaN,<NA>,[],<NA>
7,county_number,31339341,7206498,<NA>,1,99,NaN,NaN,NaN,[],99,1.912535,24132843,"[{'value': None, 'count': 7206498}, {'value': ...",<NA>,<NA>,NaN,<NA>,[],<NA>
8,date,31339341,0,<NA>,2012-01-03,2025-03-31,NaN,NaN,NaN,[],3492,10.000000,31339341,"[{'value': '2015-04-29', 'count': 35482}, {'va...",<NA>,<NA>,NaN,<NA>,[],<NA>
9,invoice_and_item_number,31339341,0,<NA>,306831300001,S444400083,NaN,NaN,NaN,[],31649878,14.054386,31339341,"[{'value': 'INV-60784400016', 'count': 324}, {...",<NA>,<NA>,NaN,<NA>,[],<NA>


### Create A BigQuery Dataset

Create a new [BigQuery Dataset](https://cloud.google.com/bigquery/docs/datasets) as a working location for this workflow:

In [11]:
%%bigquery
CREATE SCHEMA IF NOT EXISTS `statmike-mlops-349915.applied_ml_forecasting`
    OPTIONS(
        location = 'US'
    )

Query is running:   0%|          |

""


### Create A Source Table or View

This SQL query prepares the source data by selecting sales records for specific counties ('POLK', 'LINN', 'SCOTT') and ensuring that only data from stores active as of four Saturdays prior to the latest recorded date in the original dataset is included.

In [12]:
%%bigquery
CREATE OR REPLACE TABLE `statmike-mlops-349915.applied_ml_forecasting.source` AS
WITH DateRange AS (
    SELECT
        MAX(date) AS max_date
    FROM `bigquery-public-data.iowa_liquor_sales.sales`
),
ActiveStores AS (
    SELECT DISTINCT store_number
    FROM `bigquery-public-data.iowa_liquor_sales.sales`
    WHERE date = (SELECT DATE_SUB(max_date, INTERVAL 4 WEEK) FROM DateRange)
)
SELECT
    store_number,
    city,
    county,
    date,
    SUM(bottles_sold) AS total_units_sold
FROM `bigquery-public-data.iowa_liquor_sales.sales`
WHERE county IN ('POLK', 'LINN', 'SCOTT')
    AND store_number IN (SELECT store_number FROM ActiveStores)
GROUP BY store_number, date, city, county;

Query is running:   0%|          |

""


In [13]:
%%bigquery
SELECT
    county,
    city,
    store_number,
    COUNT(*) AS row_count
FROM `statmike-mlops-349915.applied_ml_forecasting.source`
GROUP BY county, city, store_number;

Query is running:   0%|          |

Downloading:   0%|          |

,county,city,store_number,row_count
0,POLK,ALTOONA,2548,939
1,POLK,ANKENY,10074,143
2,SCOTT,BETTENDORF,10065,153
3,SCOTT,BETTENDORF,2687,134
4,SCOTT,BETTENDORF,4699,249
...,...,...,...,...
110,POLK,WEST DES MOINES,5873,303
111,POLK,WEST DES MOINES,6044,81
112,POLK,WEST DES MOINES,6046,133
113,POLK,WEST DES MOINES,6119,282


### Prepare Source Table for Forecasting

This SQL query prepares the data for model training and evaluation by adding a splits column based on a date cutoff. Records with dates occurring after the date that is four Saturdays prior to the latest date in the source table are labeled as 'TEST', effectively using the end of day on that Saturday as the split point due to the behavior of the INTERVAL 4 WEEK subtraction. All earlier records are labeled as 'TRAIN', creating a distinct separation for evaluating model performance on recent data.

In [14]:
%%bigquery
CREATE OR REPLACE TABLE `statmike-mlops-349915.applied_ml_forecasting.prepped` AS
WITH DateRange AS (
    SELECT
        MAX(date) AS max_date
    FROM `statmike-mlops-349915.applied_ml_forecasting.source`
)
SELECT
    *,
    CASE
        WHEN date > DATE_SUB((SELECT max_date FROM DateRange), INTERVAL 4 WEEK) THEN 'TEST'
        ELSE 'TRAIN'
    END AS splits
FROM `statmike-mlops-349915.applied_ml_forecasting.source`

Query is running:   0%|          |

""


---
## Base-Level Forecasting With BigQuery ML's ARIMA+

Start by creating forecast for the lowest level of the hierarchy, the stores, directly.  This version create a forecast model for each time series represented by `time_series_id_col = ['county', 'city', 'store_number']` which is really just `store_number` since they are uniquely numbered.

In [15]:
%%bigquery
CREATE OR REPLACE MODEL `statmike-mlops-349915.applied_ml_forecasting.base_forecast`
OPTIONS (
    model_type = 'ARIMA_PLUS',
    time_series_timestamp_col = 'date',
    time_series_data_col = 'total_units_sold',
    time_series_id_col = ['county', 'city', 'store_number'],
    holiday_region = 'US',
    data_frequency = 'DAILY',
    horizon = 90
) AS
SELECT * EXCEPT(splits)
FROM `statmike-mlops-349915.applied_ml_forecasting.prepped`
WHERE splits = 'TRAIN'

Query is running:   0%|          |

""


### Review Forecast Values

Reference for [`ML.FORECAST`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-forecast)

Calculates the forecasted value per timestamp and provides standard error and prediction intervals at the requested confidence level.

The default `horizon` is 3 so this parameter may need to be set to the size used at training.

In [16]:
%%bigquery
SELECT *
FROM ML.FORECAST(
    MODEL `statmike-mlops-349915.applied_ml_forecasting.base_forecast`,
    STRUCT(1 AS horizon, 0.95 AS confidence_level)
)
ORDER BY county, city, store_number, forecast_timestamp

Query is running:   0%|          |

Downloading:   0%|          |

,store_number,city,county,forecast_timestamp,forecast_value,standard_error,confidence_level,prediction_interval_lower_bound,prediction_interval_upper_bound,confidence_interval_lower_bound,confidence_interval_upper_bound
0,10208,CEDAR RAPIDS,LINN,2025-03-04 00:00:00+00:00,385.608940,30.239660,0.95,326.446314,444.771567,326.446314,444.771567
1,10300,CEDAR RAPIDS,LINN,2025-03-04 00:00:00+00:00,231.328704,34.298202,0.95,164.225708,298.431699,164.225708,298.431699
2,10523,CEDAR RAPIDS,LINN,2025-03-04 00:00:00+00:00,704.094643,59.387464,0.95,587.905558,820.283728,587.905558,820.283728
3,2508,CEDAR RAPIDS,LINN,2025-03-04 00:00:00+00:00,987.328942,89.450017,0.95,812.323733,1162.334152,812.323733,1162.334152
4,2552,CEDAR RAPIDS,LINN,2025-03-04 00:00:00+00:00,763.623463,133.187624,0.95,503.047459,1024.199467,503.047459,1024.199467
...,...,...,...,...,...,...,...,...,...,...,...
110,10241,LE CLAIRE,SCOTT,2025-03-04 00:00:00+00:00,16.790718,39.429072,0.95,-60.350610,93.932046,-60.350610,93.932046
111,5156,LE CLAIRE,SCOTT,2025-03-04 00:00:00+00:00,138.958042,3.985834,0.95,131.159925,146.756159,131.159925,146.756159
112,5656,LE CLAIRE,SCOTT,2025-03-04 00:00:00+00:00,107.735031,23.401455,0.95,61.951065,153.518998,61.951065,153.518998
113,4911,PRINCETON,SCOTT,2025-03-04 00:00:00+00:00,102.302179,4.457706,0.95,93.580863,111.023495,93.580863,111.023495


### Forecast Evaluation

Reference for [`ML.EVALUATE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-evaluate) for `model_type = 'ARIMA_PLUS'`

The metrics returned depend on if input (test) data is provided and if `perform_aggregation` is `True` or `False`.  If `False` then metrics per timestamp are provide, and if `True` then metrics per `time_series_id_col` are provided.

In [17]:
%%bigquery
SELECT *
FROM ML.EVALUATE(
    MODEL `statmike-mlops-349915.applied_ml_forecasting.base_forecast`,
    (
        SELECT *
        FROM `statmike-mlops-349915.applied_ml_forecasting.prepped`
        WHERE splits = 'TEST'
        ORDER BY county, city, store_number
    ),
    STRUCT(TRUE AS perform_aggregation)
)
ORDER BY county, city, store_number

Query is running:   0%|          |

Downloading:   0%|          |

,store_number,city,county,mean_absolute_error,mean_squared_error,root_mean_squared_error,mean_absolute_percentage_error,symmetric_mean_absolute_percentage_error
0,10208,CEDAR RAPIDS,LINN,62.321366,5750.049473,75.829081,18.768092,17.447772
1,10300,CEDAR RAPIDS,LINN,71.857748,9489.803982,97.415625,12.280133,13.883940
2,10523,CEDAR RAPIDS,LINN,123.052960,22244.916402,149.147298,16.257461,16.740826
3,2508,CEDAR RAPIDS,LINN,226.674811,86273.541266,293.723580,40.504712,29.251267
4,2552,CEDAR RAPIDS,LINN,596.206716,448791.412325,669.918960,18200.240978,130.556931
...,...,...,...,...,...,...,...,...
107,10241,LE CLAIRE,SCOTT,59.844502,5062.056257,71.148129,49.464377,36.248738
108,5156,LE CLAIRE,SCOTT,125.480395,15745.329418,125.480395,47.173081,61.733994
109,5656,LE CLAIRE,SCOTT,107.170796,14526.968139,120.527873,55.909397,79.487944
110,4911,PRINCETON,SCOTT,235.105901,55274.784582,235.105901,75.596753,121.535015


---
## Hierarchical Forecasting With BigQuery ML's ARIMA+

Provide the hierarchy when creating the forecast with `hierarchical_time_series_cols = ['county', 'city', 'store_number']`.  This will automatically create an overall forecast as well so no need to specify the `state` which is always 'Iowa' in this data.

In [18]:
%%bigquery
CREATE OR REPLACE MODEL `statmike-mlops-349915.applied_ml_forecasting.hierarchical_forecast`
OPTIONS (
    model_type = 'ARIMA_PLUS',
    time_series_timestamp_col = 'date',
    time_series_data_col = 'total_units_sold',
    time_series_id_col = ['county', 'city', 'store_number'],
    hierarchical_time_series_cols = ['county', 'city', 'store_number'],
    holiday_region = 'US',
    data_frequency = 'DAILY',
    horizon = 90
) AS
SELECT * EXCEPT(splits)
FROM `statmike-mlops-349915.applied_ml_forecasting.prepped`
WHERE splits = 'TRAIN'

Query is running:   0%|          |

""


### Review Forecast Values

Reference for [`ML.FORECAST`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-forecast)

Calculates the forecasted value per timestamp and provides standard error and prediction intervals at the requested confidence level.

The default `horizon` is 3 so this parameter may need to be set to the size used at training.

In [19]:
%%bigquery
SELECT *
FROM ML.FORECAST(
    MODEL `statmike-mlops-349915.applied_ml_forecasting.hierarchical_forecast`,
    STRUCT(1 AS horizon, 0.95 AS confidence_level)
)
ORDER BY county, city, store_number, forecast_timestamp

Query is running:   0%|          |

Downloading:   0%|          |

,store_number,city,county,forecast_timestamp,forecast_value,standard_error,confidence_level,prediction_interval_lower_bound,prediction_interval_upper_bound,confidence_interval_lower_bound,confidence_interval_upper_bound
0,None,None,None,2025-03-04 00:00:00+00:00,61328.676967,7795.136399,0.95,46077.819283,76579.534651,46077.819283,76579.534651
1,None,None,LINN,2025-03-04 00:00:00+00:00,16974.738077,1569.772687,0.95,13903.543602,20045.932552,13903.543602,20045.932552
2,None,CEDAR RAPIDS,LINN,2025-03-04 00:00:00+00:00,14749.377260,1134.597054,0.95,12529.585673,16969.168848,12529.585673,16969.168848
3,10208,CEDAR RAPIDS,LINN,2025-03-04 00:00:00+00:00,385.608940,30.239660,0.95,326.446314,444.771567,326.446314,444.771567
4,10300,CEDAR RAPIDS,LINN,2025-03-04 00:00:00+00:00,231.328704,34.298202,0.95,164.225708,298.431699,164.225708,298.431699
...,...,...,...,...,...,...,...,...,...,...,...
132,5656,LE CLAIRE,SCOTT,2025-03-04 00:00:00+00:00,107.735031,23.401455,0.95,61.951065,153.518998,61.951065,153.518998
133,None,PRINCETON,SCOTT,2025-03-04 00:00:00+00:00,102.302179,4.457706,0.95,93.580863,111.023495,93.580863,111.023495
134,4911,PRINCETON,SCOTT,2025-03-04 00:00:00+00:00,102.302179,4.457706,0.95,93.580863,111.023495,93.580863,111.023495
135,None,WALCOTT,SCOTT,2025-03-04 00:00:00+00:00,377.546652,8.179839,0.95,361.543140,393.550164,361.543140,393.550164


### Forecast Evaluation

Reference for [`ML.EVALUATE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-evaluate) for `model_type = 'ARIMA_PLUS'`

The metrics returned depend on if input (test) data is provided and if `perform_aggregation` is `True` or `False`.  If `False` then metrics per timestamp are provide, and if `True` then metrics per `time_series_id_col` are provided.

In [20]:
%%bigquery
SELECT *
FROM ML.EVALUATE(
    MODEL `statmike-mlops-349915.applied_ml_forecasting.hierarchical_forecast`,
    (
        SELECT *
        FROM `statmike-mlops-349915.applied_ml_forecasting.prepped`
        WHERE splits = 'TEST'
        ORDER BY county, city, store_number
    ),
    STRUCT(TRUE AS perform_aggregation)
)
ORDER BY county, city, store_number

Query is running:   0%|          |

Downloading:   0%|          |

,store_number,city,county,mean_absolute_error,mean_squared_error,root_mean_squared_error,mean_absolute_percentage_error,symmetric_mean_absolute_percentage_error
0,10208,CEDAR RAPIDS,LINN,62.321366,5750.049473,75.829081,18.768092,17.447772
1,10300,CEDAR RAPIDS,LINN,71.857748,9489.803982,97.415625,12.280133,13.883940
2,10523,CEDAR RAPIDS,LINN,123.052960,22244.916402,149.147298,16.257461,16.740826
3,2508,CEDAR RAPIDS,LINN,226.674811,86273.541266,293.723580,40.504712,29.251267
4,2552,CEDAR RAPIDS,LINN,596.206716,448791.412325,669.918960,18200.240978,130.556931
...,...,...,...,...,...,...,...,...
107,10241,LE CLAIRE,SCOTT,59.844502,5062.056257,71.148129,49.464377,36.248738
108,5156,LE CLAIRE,SCOTT,125.480395,15745.329418,125.480395,47.173081,61.733994
109,5656,LE CLAIRE,SCOTT,107.170796,14526.968139,120.527873,55.909397,79.487944
110,4911,PRINCETON,SCOTT,235.105901,55274.784582,235.105901,75.596753,121.535015


---
## Compare Base-Level Forecasting To Hierarchical Forecasting

The hierarchical forecast aggregates base-level forecasts. For each time point and level of the hierarchy (store, city, county, state), the hierarchical forecast value is derived by summing the base-level forecasts of the constituent lower-level entities. Specifically, store forecasts are summed to obtain city forecasts, city forecasts are summed for county forecasts, and county forecasts are summed for the state forecast.

### Review Base-Level: `store_number`

This query verifies that the base-level forecast for a specific time series (store_number '10523') matches the corresponding lowest-level forecast within the hierarchical model across the 10-day forecast horizon.

In [21]:
%%bigquery
WITH
baseForecast AS (
    SELECT store_number, city, county, forecast_timestamp, forecast_value AS base_forecast_value
    FROM ML.FORECAST(
        MODEL `statmike-mlops-349915.applied_ml_forecasting.base_forecast`,
        STRUCT(10 AS horizon, 0.95 AS confidence_level)
    )
    WHERE store_number = '10523'
),
hierarchicalForecast AS (
    SELECT store_number, city, county, forecast_timestamp, forecast_value AS hierarchical_forecast_value
    FROM ML.FORECAST(
        MODEL `statmike-mlops-349915.applied_ml_forecasting.hierarchical_forecast`,
        STRUCT(10 AS horizon, 0.95 AS confidence_level)
    )
    WHERE store_number = '10523'
)
SELECT bf.*, hf.hierarchical_forecast_value
FROM baseForecast bf
JOIN hierarchicalForecast hf
ON bf.store_number = hf.store_number AND bf.forecast_timestamp = hf.forecast_timestamp

Query is running:   0%|          |

Downloading:   0%|          |

,store_number,city,county,forecast_timestamp,base_forecast_value,hierarchical_forecast_value
0,10523,CEDAR RAPIDS,LINN,2025-03-04 00:00:00+00:00,704.094643,704.094643
1,10523,CEDAR RAPIDS,LINN,2025-03-05 00:00:00+00:00,701.351263,701.351263
2,10523,CEDAR RAPIDS,LINN,2025-03-06 00:00:00+00:00,699.196080,699.196080
3,10523,CEDAR RAPIDS,LINN,2025-03-07 00:00:00+00:00,697.643416,697.643416
4,10523,CEDAR RAPIDS,LINN,2025-03-08 00:00:00+00:00,696.524826,696.524826
5,10523,CEDAR RAPIDS,LINN,2025-03-09 00:00:00+00:00,695.718957,695.718957
6,10523,CEDAR RAPIDS,LINN,2025-03-10 00:00:00+00:00,695.138383,695.138383
7,10523,CEDAR RAPIDS,LINN,2025-03-11 00:00:00+00:00,694.720119,694.720119
8,10523,CEDAR RAPIDS,LINN,2025-03-12 00:00:00+00:00,694.418787,694.418787
9,10523,CEDAR RAPIDS,LINN,2025-03-13 00:00:00+00:00,694.201698,694.201698


### Review **city** from: state > county > city > store_number

This query validates the hierarchical aggregation at the city level by comparing the direct 10-day forecast for 'CEDAR RAPIDS' with the sum of the individual 10-day store-level forecasts within the same city. The output shows the city-level forecast alongside the sum of its constituent store forecasts for each time point in the 10-day horizon, allowing for a direct comparison of the aggregated values.

This SQL query verifies the hierarchical forecast aggregation at the city level ('CEDAR RAPIDS') by performing the following steps:

- Get City Forecast: Retrieves the direct 10-day forecast for 'CEDAR RAPIDS' (where store_number is null).
- Sum Store Forecasts: Calculates the sum of the 10-day forecasts for all individual stores within 'CEDAR RAPIDS'.
- Compare Forecasts: Joins the city forecast with the summed store forecasts based on the forecast timestamp.
- View Results: Shows the city-level forecast alongside the aggregated store-level forecast for each of the 10 forecast time points.

In [22]:
%%bigquery
WITH
hierarchicalForecast AS (
    SELECT store_number, city, county, forecast_timestamp, forecast_value AS hierarchical_forecast_value
    FROM ML.FORECAST(
        MODEL `statmike-mlops-349915.applied_ml_forecasting.hierarchical_forecast`,
        STRUCT(10 AS horizon, 0.95 AS confidence_level)
    )
    WHERE city = 'CEDAR RAPIDS'
        AND store_number IS null
),
sumStore AS (
    SELECT forecast_timestamp, SUM(forecast_value) AS sum_hierarchical_forecast_value
    FROM ML.FORECAST(
        MODEL `statmike-mlops-349915.applied_ml_forecasting.hierarchical_forecast`,
        STRUCT(10 AS horizon, 0.95 AS confidence_level)
    )
    WHERE city = 'CEDAR RAPIDS'
        AND store_number IS NOT null
    GROUP BY forecast_timestamp
)
SELECT hf.*, ss.sum_hierarchical_forecast_value
FROM hierarchicalForecast hf
JOIN sumStore ss
on hf.forecast_timestamp = ss.forecast_timestamp
ORDER BY hf.forecast_timestamp
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

,store_number,city,county,forecast_timestamp,hierarchical_forecast_value,sum_hierarchical_forecast_value
0,None,CEDAR RAPIDS,LINN,2025-03-04 00:00:00+00:00,14749.377260,14749.377260
1,None,CEDAR RAPIDS,LINN,2025-03-05 00:00:00+00:00,15528.064720,15528.064720
2,None,CEDAR RAPIDS,LINN,2025-03-06 00:00:00+00:00,14946.682108,14946.682108
3,None,CEDAR RAPIDS,LINN,2025-03-07 00:00:00+00:00,14837.692819,14837.692819
4,None,CEDAR RAPIDS,LINN,2025-03-08 00:00:00+00:00,15121.165727,15121.165727
5,None,CEDAR RAPIDS,LINN,2025-03-09 00:00:00+00:00,15219.037132,15219.037132
6,None,CEDAR RAPIDS,LINN,2025-03-10 00:00:00+00:00,15348.382089,15348.382089
7,None,CEDAR RAPIDS,LINN,2025-03-11 00:00:00+00:00,15735.421929,15735.421929
8,None,CEDAR RAPIDS,LINN,2025-03-12 00:00:00+00:00,16070.073994,16070.073994
9,None,CEDAR RAPIDS,LINN,2025-03-13 00:00:00+00:00,14940.171444,14940.171444


### Review **county** from: state > county > city > store_number

This query validates the hierarchical aggregation at the county level by comparing the direct 10-day forecast for 'LINN' with the sum of the individual 10-day city-level forecasts within the same county. The output shows the county-level forecast alongside the sum of its constituent city forecasts for each time point in the 10-day horizon, allowing for a direct comparison of the aggregated values.

This SQL query verifies the hierarchical forecast aggregation at the county level ('LINN') by performing the following steps:

- Get County Forecast: Retrieves the direct 10-day forecast for 'LINN' (where city is null).
- Sum City Forecasts: Calculates the sum of the 10-day forecasts for all individual cities within 'LINN' (where store_number is null).
- Compare Forecasts: Joins the county forecast with the summed city forecasts based on the forecast timestamp.
- View Results: Shows the county-level forecast alongside the aggregated city-level forecast for each of the 10 forecast time points.

In [23]:
%%bigquery
WITH
hierarchicalForecast AS (
    SELECT store_number, city, county, forecast_timestamp, forecast_value AS hierarchical_forecast_value
    FROM ML.FORECAST(
        MODEL `statmike-mlops-349915.applied_ml_forecasting.hierarchical_forecast`,
        STRUCT(10 AS horizon, 0.95 AS confidence_level)
    )
    WHERE county = 'LINN'
        AND city IS null
),
sumCity AS (
    SELECT forecast_timestamp, SUM(forecast_value) AS sum_hierarchical_forecast_value
    FROM ML.FORECAST(
        MODEL `statmike-mlops-349915.applied_ml_forecasting.hierarchical_forecast`,
        STRUCT(10 AS horizon, 0.95 AS confidence_level)
    )
    WHERE county = 'LINN'
        AND city IS NOT null
        AND store_number IS null
    GROUP BY forecast_timestamp
)
SELECT hf.*, ss.sum_hierarchical_forecast_value
FROM hierarchicalForecast hf
JOIN sumCity ss
on hf.forecast_timestamp = ss.forecast_timestamp
ORDER BY hf.forecast_timestamp
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

,store_number,city,county,forecast_timestamp,hierarchical_forecast_value,sum_hierarchical_forecast_value
0,None,None,LINN,2025-03-04 00:00:00+00:00,16974.738077,16974.738077
1,None,None,LINN,2025-03-05 00:00:00+00:00,18631.455486,18631.455486
2,None,None,LINN,2025-03-06 00:00:00+00:00,19029.947041,19029.947041
3,None,None,LINN,2025-03-07 00:00:00+00:00,16749.996700,16749.996700
4,None,None,LINN,2025-03-08 00:00:00+00:00,16690.194840,16690.194840
5,None,None,LINN,2025-03-09 00:00:00+00:00,16427.390011,16427.390011
6,None,None,LINN,2025-03-10 00:00:00+00:00,16229.173461,16229.173461
7,None,None,LINN,2025-03-11 00:00:00+00:00,17421.910830,17421.910830
8,None,None,LINN,2025-03-12 00:00:00+00:00,18846.634967,18846.634967
9,None,None,LINN,2025-03-13 00:00:00+00:00,18824.948271,18824.948271


### Review **state** from: state > county > city > store_number

This query validates the hierarchical aggregation at the state level by comparing the direct 10-day overall state-level forecast (identified by store_number, city, and county being NULL) with the sum of the individual 10-day county-level forecasts. The output shows the state-level forecast alongside the sum of its constituent county forecasts for each time point in the 10-day horizon, allowing for a direct comparison of the aggregated values.

This SQL query verifies the hierarchical forecast aggregation at the state level by performing the following steps:

- Get State Forecast: Retrieves the direct 10-day overall state-level forecast (where county is null).
- Sum County Forecasts: Calculates the sum of the 10-day forecasts for all individual counties (where city and store_number are null).
- Compare Forecasts: Joins the state forecast with the summed county forecasts based on the forecast timestamp.
- View Results: Shows the overall state-level forecast alongside the aggregated county-level forecast for each of the 10 forecast time points.

In [24]:
%%bigquery
WITH
hierarchicalForecast AS (
    SELECT store_number, city, county, forecast_timestamp, forecast_value AS hierarchical_forecast_value
    FROM ML.FORECAST(
        MODEL `statmike-mlops-349915.applied_ml_forecasting.hierarchical_forecast`,
        STRUCT(10 AS horizon, 0.95 AS confidence_level)
    )
    WHERE county IS null
),
sumCounty AS (
    SELECT forecast_timestamp, SUM(forecast_value) AS sum_hierarchical_forecast_value
    FROM ML.FORECAST(
        MODEL `statmike-mlops-349915.applied_ml_forecasting.hierarchical_forecast`,
        STRUCT(10 AS horizon, 0.95 AS confidence_level)
    )
    WHERE county IS NOT null
        AND city IS null
        AND store_number IS null
    GROUP BY forecast_timestamp
)
SELECT hf.*, ss.sum_hierarchical_forecast_value
FROM hierarchicalForecast hf
JOIN sumCounty ss
on hf.forecast_timestamp = ss.forecast_timestamp
ORDER BY hf.forecast_timestamp
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

,store_number,city,county,forecast_timestamp,hierarchical_forecast_value,sum_hierarchical_forecast_value
0,None,None,None,2025-03-04 00:00:00+00:00,61328.676967,61328.676967
1,None,None,None,2025-03-05 00:00:00+00:00,61049.510123,61049.510123
2,None,None,None,2025-03-06 00:00:00+00:00,75617.956648,75617.956648
3,None,None,None,2025-03-07 00:00:00+00:00,63653.047599,63653.047599
4,None,None,None,2025-03-08 00:00:00+00:00,66783.947079,66783.947079
5,None,None,None,2025-03-09 00:00:00+00:00,69321.825093,69321.825093
6,None,None,None,2025-03-10 00:00:00+00:00,72985.453810,72985.453810
7,None,None,None,2025-03-11 00:00:00+00:00,62414.910190,62414.910190
8,None,None,None,2025-03-12 00:00:00+00:00,62369.093733,62369.093733
9,None,None,None,2025-03-13 00:00:00+00:00,75054.145737,75054.145737


---
## Top-Down Hierarchical Forecasting - Custom Approach

The preceding verification steps demonstrate that the hierarchical forecasts are indeed generated through a **bottom-up aggregation** strategy. The forecast value at each level of the hierarchy is simply the sum of the forecast values from the level below it, starting from the base store-level forecasts up to the overall state forecast.

Next, we will develop a custom approach to the alternative **top-down disaggregation**. This involves generating a forecast for the top level (state) and then disaggregating this forecast down the hierarchy: from state to county, county to city, and finally city to store. This disaggregation will be performed by distributing the higher-level forecast proportionally among the lower-level series. Several methods exist for creating these proportions:

- Average historical proportions: Calculated as the average of historical values of the series at the bottom level relative to the historical values of the top level.
- Proportion of historical averages: Determined by the ratio of the average historical value of a bottom-level series to the average historical value of the top-level series.
- Forecast proportions: Proportions derived from independently generated forecasts for each series within the hierarchy. Notably, only the top-level forecast is used directly; the lower-level forecasts are used solely to calculate the proportions for disaggregation at each time step of the horizon.

This workflow will utilize **forecast proportions**. The two averaging methods (average historical proportions and proportion of historical averages) can suffer from reduced accuracy at lower levels because they:

- Lose granular information present in individual lower-level series.
- Fail to account for the unique characteristics and patterns of each lower-level series.
- Do not adapt to changes or evolving patterns over time.


### Generate Forecast For Each Time Series

First, generate a forecast for each time series in the hierarchy.  This query create a view on top of the prepped data that stacks each level of the hierarchy.